# Download Datasets

In [ ]:
!pip install mne

In [ ]:
# Download BDF file
!wget -O eeg_data_02.bdf https://www.dropbox.com/scl/fi/et82hv39u24ll3brk8n0l/sub-02_ses-01_task-innerspeech_eeg.bdf?rlkey=prc1yt31nk1hb66h1qml4wkti&dl=0
!wget -O eeg_data_03.bdf https://www.dropbox.com/scl/fi/252j4sc1ymcibgitg57dv/sub-03_ses-01_task-innerspeech_eeg.bdf?rlkey=26o26pc7dgucbl5y60f9avd8y&dl=0
!wget -O eeg_data_04.bdf https://www.dropbox.com/scl/fi/m5nbdcgh2jsp4fqz1vl76/sub-04_ses-01_task-innerspeech_eeg.bdf?rlkey=07j2jydunn4u7nu36zee2nngl&dl=0
!wget -O eeg_data_05.bdf https://www.dropbox.com/scl/fi/0ob6affaz1nxyzmpif77r/sub-05_ses-01_task-innerspeech_eeg.bdf?rlkey=x6morhp7kho5xhc1bl5zm40fj&dl=0
!wget -O eeg_data_06.bdf https://www.dropbox.com/scl/fi/qgelieb2o3alzsh13kkq9/sub-06_ses-01_task-innerspeech_eeg.bdf?rlkey=suwr5st63vx43o6xzguyait0q&dl=0
!wget -O eeg_data_07.bdf https://www.dropbox.com/scl/fi/ln1tbvlcsdw4examr4uwo/sub-07_ses-01_task-innerspeech_eeg.bdf?rlkey=rktp9di84qv6ef8khk6iom84c&dl=0

In [ ]:
import mne

file_path = 'eeg_data_02.bdf'

# Read the .bdf file
raw = mne.io.read_raw_bdf(file_path, preload=True)

# Plot the data (this requires a graphical interface)
raw.plot()

# Data Pre-Processing

In [ ]:
import mne

# Function to process each dataset
def process_dataset(file_path):
    # Read BDF File
    raw = mne.io.read_raw_bdf(file_path, preload=True)
    # Re-reference
    raw.set_eeg_reference(['EXG1', 'EXG2']) # NODE CHANGES HERE: 'EXG1', 'EXG2', EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8'
    # Apply bandpass filter (8 to 30 Hz) and Notch filter at 50 Hz
    raw.filter(l_freq=8, h_freq=30, fir_design='firwin')
    raw = mne.io.Raw.notch_filter(raw, freqs=50)
    # Decimate the data to reduce sampling rate to around 254 Hz
    raw.resample(254)
    # Epoching: Define events and epochs based on experimental design
    events = mne.find_events(raw, initial_event=True, consecutive=True, min_duration=0.002)
    event_id = dict(Down=32, Right=34) # dict(Up=31, Down=32, Left=33, Right=34)
    picks_vir = mne.pick_types(raw.info, eeg=True, include=['EXG1', 'EXG2'], stim=False)
    epochs = mne.Epochs(raw, events, event_id=event_id, tmin=-0.5, tmax=4, picks=picks_vir, preload=True, decim = 3, baseline=None)
    print("*******************************************************************************************************************************************************")
    return epochs

# Download and process each dataset
epochs_02 = process_dataset('eeg_data_02.bdf')
epochs_03 = process_dataset('eeg_data_03.bdf')
epochs_04 = process_dataset('eeg_data_04.bdf')
epochs_05 = process_dataset('eeg_data_05.bdf')
epochs_06 = process_dataset('eeg_data_06.bdf')
epochs_07 = process_dataset('eeg_data_07.bdf')

# Combine epochs from all datasets
all_epochs = mne.concatenate_epochs([epochs_02, epochs_03, epochs_04, epochs_05, epochs_06, epochs_07])

In [ ]:
# Feature Extraction

import numpy as np

# Compute PSD using the compute_psd method
spectrum = all_epochs.compute_psd(method='welch')
data, freqs = spectrum.get_data(return_freqs=True)

# Skip conversion to dB, use the power values directly
psds_power = data

# Mean PSD values across frequencies for each channel
features = psds_power.mean(axis=1)

In [ ]:
# Data Normalization

from sklearn.preprocessing import StandardScaler

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

In [ ]:
# Dimensionality Reduction

from sklearn.decomposition import PCA

# Apply PCA for dimensionality reduction
pca = PCA(n_components=0.95)  # Keep 95% of variance
X_pca = pca.fit_transform(X_scaled)

# Machine Learning Models

In [ ]:
# LDA Model Building
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split

# Preparing labels for classification
y = all_epochs.events[:, -1]  # Assuming last column of events array contains the labels

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Class Splits
class_priors = [0.5, 0.5];

# Create LDA model
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto', priors=class_priors) #try solver='eigen'
lda.fit(X_train, y_train)

# -------------------------------------------------------------------------------------------------------

# Model Validation and Evaluation

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

# Predict on test data
y_pred_lda = lda.predict(X_test)

# Evaluate the LDA model
accuracy_lda = accuracy_score(y_test, y_pred_lda)
conf_matrix_lda = confusion_matrix(y_test, y_pred_lda)
class_report_lda = classification_report(y_test, y_pred_lda)

print(f"LDA Accuracy: {accuracy_lda}\n")
print(f"LDA Confusion Matrix:\n{conf_matrix_lda}\n")
print(f"LDA Classification Report:\n{class_report_lda}\n")

In [ ]:
# Random Forest Model Building
from sklearn.ensemble import RandomForestClassifier

# Preparing labels for classification
y = all_epochs.events[:, -1]  # Assuming last column of events array contains the labels

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Create RF Model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# -------------------------------------------------------------------------------------------------------

# Model Validation and Evaluation

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

# Predict on test data
y_pred_rf = rf.predict(X_test)

# Evaluate the Random Forest model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
class_report_rf = classification_report(y_test, y_pred_rf)

print(f"Random Forest Accuracy: {accuracy_rf}\n")
print(f"Random Forest Confusion Matrix:\n{conf_matrix_rf}\n")
print(f"Random Forest Classification Report:\n{class_report_rf}\n")

# Finding Best RF Parameters

In [ ]:
from sklearn.model_selection import GridSearchCV

# Parameters to tune
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Grid Search with cross-validation
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5)
rf_grid.fit(X_train, y_train)

# Use the best estimator for predictions
y_pred_rf = rf_grid.best_estimator_.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Print the best parameters and best score
print("Best Parameters:", rf_grid.best_params_)
print("Best Training Score:", rf_grid.best_score_)

# Evaluate on the test set
y_pred_rf = rf_grid.best_estimator_.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
class_report_rf = classification_report(y_test, y_pred_rf)

# Print test set evaluation metrics
print(f"Random Forest Test Set Accuracy: {accuracy_rf}")
print(f"Random Forest Confusion Matrix on Test Set:\n{conf_matrix_rf}")
print(f"Random Forest Classification Report on Test Set:\n{class_report_rf}")


# CNN Prototype

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Reshaping the data for CNN input

# Accessing the data from the Epochs object
data = all_epochs.get_data()
# Data is a 3D array of shape (n_epochs, n_channels, n_times)
n_epochs, n_channels, n_times = data.shape

X_cnn = data.reshape((n_epochs, n_channels, n_times, 1))

# Convert y to a continuous range starting from 0
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert the encoded labels to categorical (one-hot encoding)
y_categorical = to_categorical(y_encoded)

# Splitting the data
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X_cnn, y_categorical, test_size=0.3, random_state=42)

# CNN Model
model = Sequential()
model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(n_channels, n_times, 1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(len(np.unique(y)), activation='softmax')) # Number of classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_cnn, y_train_cnn, validation_data=(X_test_cnn, y_test_cnn), epochs=10)

# Evaluate the model
accuracy_cnn = model.evaluate(X_test_cnn, y_test_cnn)[1]
print(f"CNN Model Accuracy: {accuracy_cnn}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Make predictions
y_pred_cnn = model.predict(X_test_cnn)
# Convert predictions from one-hot encoded back to labels
y_pred_labels = np.argmax(y_pred_cnn, axis=1)

# Convert test labels from one-hot encoded back to labels
y_test_labels = np.argmax(y_test_cnn, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test_labels, y_pred_labels)
print(conf_matrix)

# Print classification report
print(classification_report(y_test_labels, y_pred_labels))